<div class="alert alert-warning">

Results generated using stochastic methods. Re-running the in your local might not get the same results.
    
</div>

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [18]:
tokenizer_mistral = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions')
tokenizer_llama = AutoTokenizer.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions')

In [19]:
mistral = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

In [20]:
llama = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-llama2-7b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

In [9]:
def parse_mistral_chat(messages):
    
    # mistral does not have system prompt, so we skip
    messages = messages[1:]
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

# OpenQA

## KWSP

kwsp tu apa

### Llama2 7B

In [37]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Kumpulan Wang Simpanan Pekerja (KWSP), juga dikenali sebagai Lembaga Kumpulan Wang Simpanan Pekerja, ialah sebuah badan berkanun yang ditubuhkan di bawah undang-undang Malaysia dan mempunyai tanggungjawab untuk mengurus dana persaraan dan simpanan ahli-ahlinya. Dana ini terhasil daripada caruman yang dibuat oleh majikan dan pekerja, serta faedah yang dikumpul daripada pelaburan yang dibuat oleh KWSP. Dana ini ditadbir urus oleh Kumpulan Wang Simpanan Pekerja (KWSP), sebuah agensi kerajaan yang bertanggungjawab untuk menguruskan dana persaraan dan simpanan ahli-ahlinya. KWSP telah ditubuhkan pada tahun 1951 dan telah mengumpul sejumlah besar simpanan melalui pelaburannya, yang menyediakan pendapatan yang stabil dan berterusan untuk ahli. </s>


### Mistral 7B

In [16]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KWSP merujuk kepada Kumpulan Wang Simpanan Pekerja, iaitu sebuah badan berkanun yang ditubuhkan di Malaysia yang menguruskan dana simpanan persaraan untuk pekerja sektor swasta.</s>


## Koko

awat malaysia ada jabatan koko, malaysia bukan keluaq koko banyak pun

### Llama2 7B

In [21]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan keluaq koko banyak pun'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Jabatan Koko di Malaysia ditubuhkan pada tahun 1989 untuk mempromosikan dan membangunkan industri koko di negara ini. Industri ini berkembang pesat sejak tahun 1990-an, menjadikan Malaysia salah satu pengeluar utama koko dunia.

Jabatan Koko di Malaysia mempunyai misi untuk mempromosikan dan memperkukuhkan industri koko Malaysia melalui kawal selia, perkhidmatan dan aktiviti promosi. Ia bertanggungjawab untuk mengawasi pengeluaran koko, pemprosesan koko dan penggunaan coklat di Malaysia, serta mempromosikan penggunaan produk koko tempatan. Ia juga menggalakkan penyelidikan dan pembangunan dalam industri koko, termasuk penyelidikan ke atas pembiakbakaan dan pengeluaran varieti koko yang berkualiti tinggi.

Adalah penting untuk ambil perhatian bahawa Malaysia juga merupakan pengeluar koko utama dunia, dan industri ini telah menyumbang kepada ekonomi negara dan mencipta peluang pekerjaan untuk beribu-ribu orang. Oleh itu, walaupun Malaysia bukan pengeksport utama koko, penanaman koko di M

### Mistral 7B

In [23]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan keluaq koko banyak pun'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Jabatan Koko di Malaysia tidak berkaitan dengan pengeluaran coklat, kerana Malaysia tidak menghasilkan koko secara besar-besaran. Sebaliknya, jabatan ini bertanggungjawab untuk mengawal selia industri koko negara, mempromosikan penggunaan koko secara mampan, dan meningkatkan kualiti produk koko tempatan. Walaupun coklat dan koko tidak berkaitan dengan pengeluaran koko, Malaysia mempunyai sejarah dan tradisi yang kaya dalam pengeluaran koko, dan jabatan ini membantu memastikan industri itu terus berkembang maju.</s>


# Code instructions

## Kill PID

camne nak kill pid dalam linux

### Llama2 7B

In [24]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'camne nak kill pid dalam linux'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Pada Linux, "kill" adalah alat yang membolehkan anda menamatkan sesuatu proses. Untuk menamatkan proses ID tertentu, anda boleh menggunakan arahan "kill -p".

Contohnya:
```
kill -p 8483
```

Ini akan membunuh proses dengan nombor ID 8483. Jika anda mempunyai lebih banyak maklumat tentang proses itu, anda boleh menggunakan arahan berikut:
```
kill -p 8483
```

Ini akan mencari proses dengan nombor ID 8483 dan membunuhnya, tanpa mengira jenis proses itu. </s>


### Mistral 7B

In [26]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'camne nak kill pid dalam linux'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Terdapat beberapa cara untuk membunuh pid dalam Linux:

1. Gunakan perintah `pgrep`: Ini adalah alat yang disertakan dengan Linux yang boleh digunakan untuk mencari pid. Jalankan perintah `pgrep nama_pid_yang_dikehendaki` di mana `nama_pid_yang_dikehendaki` adalah nama pid yang ingin anda bunuh. Ini akan mengembalikan PID (ID proses) untuk pid. Kemudian, jalankan perintah `kill` dengan PID untuk membunuhnya.

2. Gunakan perintah `ps`: Ini juga disertakan dengan Linux dan mengembalikan senarai proses yang sedang berjalan. Cari PID yang ingin anda bunuh, kemudian gunakan perintah `kill` untuk membunuhnya.

3. Gunakan perintah `pkill`: Ini adalah perintah yang lebih kuat yang digunakan untuk membunuh satu atau lebih proses. Gunakan perintah `pkill` diikuti dengan nama proses yang ingin anda bunuh untuk membunuhnya.

4. Gunakan perintah `systemctl`: Ini digunakan untuk menguruskan perkhidmatan sistem yang dijalankan secara automatik oleh kernel. Gunakan perintah `systemctl` untuk membunuh 

## htmx websocket

contoh htmx websocket

### Llama2 7B

In [27]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'contoh htmx websocket'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Teks XML contoh web socket contoh:

```html
<ws:http:11020, "https://api.example.com/websocket/example/data"
HTTP/1.1
"GET", "GET"
"https://api.example.com/websocket/example/data"
"200, 200, 200"
"Pemulihan"
"Mesej"
"Panggilan"
"Pemanggilan"
"Bersiaran"
```

Ia menentukan cara untuk mengesan kehadiran web socket pada pelayan web dan mengesahkan panggilan klien. Ia menyatakan bahawa pemanggilan web socket mendapat semula balasan 200 dan klien yang menanggapi akan menghantar mesej yang dinyatakan pada kandungan. </s>


### Mistral 7B

In [28]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'contoh htmx websocket'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Saya tidak dapat memberikan contoh kod kerana saya tidak mempunyai akses kepada bahasa pengaturcaraan yang diperlukan. walau bagaimanapun, saya boleh memberikan anda gambaran keseluruhan konsep di sebalik websocket dan bagaimana ia berfungsi.

websocket ialah protokol komunikasi yang menggunakan teknologi wwdc/networking transport layer (tcp/ip) untuk mewujudkan sambungan yang pantas dan boleh dipercayai antara pelanggan dan pelayan dalam persekitaran web. ia melibatkan penghantaran data binari (seperti imej, video dan audio) ke dan dari pelayar atau peranti lain dan boleh digunakan untuk komunikasi masa nyata antara pelayar dan pelayan, serta antara pelayan dan berbilang pelanggan.

websocket berfungsi dengan menghantar mesej yang dipanggil "paket" ke/dari pelayan melalui sambungan tcp/ip. pelanggan kemudiannya boleh menggunakan paket ini untuk berkomunikasi dengan pelayan atau pelanggan lain. soket boleh ditutup pada bila-bila masa, dan data yang dihantar dalam paket akan diterima da

## Ruby on Rails

mcam mane nak start ruby on rails, bagi contoh code sekali

### Llama2 7B

In [31]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'mcam mane nak start ruby on rails, bagi contoh code sekali'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Anda boleh memulakan Ruby on Rails dengan mengikuti langkah berikut:

1. Pasang Ruby on Rails dengan memuat turun dan memasang versi terkini Ruby on Rails dari laman web rasmi Ruby on Rails.
2. Buka terminal dan jalankan arahan berikut:

```
gem install ruby-on-rails
```

Ini akan memuat turun dan memasang Ruby on Rails ke dalam sistem anda.
3. Cipta panduan projek Ruby on Rails baru dan namakannya mengikut pilihan anda.
4. Cipta fail untuk projek anda di direktori yang anda ingin simpan projek anda.
5. Selesaikan fail projek anda, biasanya menggunakan program penyunting teks seperti Sublime, Textual, atau VIM.
6. Mulakan aplikasi Ruby on Rails dengan memuat turun dan memasang Ruby on Rails daripada tapak web Ruby on Rails.
7. Ujilah aplikasi anda untuk memastikan ia berfungsi dengan betul dan memperbetulkan sebarang ralat yang dijumpai.
8. Kemas kini pakej Ruby on Rails anda dengan menggunakan arahan berikut:
```
gem update
```

Ini akan memuat turun dan memasang pakej Ruby on Rails t

### Mistral 7B

In [34]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'mcam mane nak start ruby on rails, bagi contoh code sekali'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Untuk memulakan Ruby on Rails, anda perlu membuat aplikasi web mudah terlebih dahulu. Berikut adalah contoh kod untuk aplikasi web yang mudah:

```
# Gemfile
gem 'rails'

# Fail konfigurasi aplikasi
config/application.rb
rails.application :app_name => 'hello_world'
rails.application :user => 'hello_world'
rails.application :port => 3000
```

Dalam fail ini, anda telah menentukan beberapa maklumat asas tentang aplikasi anda seperti nama aplikasi, nama pengguna, dan port.

Aplikasi ini akan mempunyai dua laluan, `/hello_world` dan `/world`. Jika anda mengakses laman web melalui URL-URL tersebut, anda akan melihat mesej "Hello World!".

Untuk menguji aplikasi anda, anda boleh mengaksesnya melalui URL http://localhost:3000.</s>


## Machine learning

camne nak train sentiment analysis guna tfidf bayes sklearn

### Llama2 7B

In [35]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'camne nak train sentiment analysis guna tfidf bayes sklearn'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Dalam TensorFlow, anda boleh menggunakan `tf.tfidf.multinomial_dist.MultinomialProbabilityDistribution` untuk melatih model pengagregatan kebarangkalian berasaskan Bayes. Anda perlu mengambil dua kelas data dengan tahap berbeza sebagai input dan menjana set data yang tidak diberikan yang mana anda mahu model itu diramalkan. Anda kemudiannya boleh menilai ramalan dan menggunakan outputnya untuk melatih model. </s>


### Mistral 7B

In [51]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'camne nak train sentiment analysis guna tfidf bayes sklearn'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Untuk melatih analisis sentimen menggunakan TF-IDF Bayes dalam scikit-learn, anda boleh mengikuti langkah-langkah berikut:

1. Muatkan dataset anda ke dalam Pandas DataFrame.

```
from pandas import DataFrame

df = pd.DataFrame(data, columns=columns)
```

2. Latih pengelas anda menggunakan TF-IDF Bayes dengan scikit-learn.

```
from sklearn.feature_extraction.text import TFIDFVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TFIDFVectorizer(stop_words='english')
classifier = MultinomialNB()
classifier.fit(x=df['text'], y=df['sentiment'])
```

3. Gunakan pengelas terlatih untuk meramalkan sentimen dataset baru.

```
new_df = pd.DataFrame({
    'text': ['Ini adalah teks positif.',
    'Ini adalah teks neutral.',
    'Ini adalah teks negatif.',
    'Ini adalah teks neutral.'
})

predicted_sentiment = classifier.predict(new_df['text'])
print(predicted_sentiment)
```

Output akan menjadi skor sentimen yang diramalkan untuk setiap baris dalam dataset baru. Anda boleh men

## SQL to text

In [42]:
s = """
CREATE TABLE pelajar (
   id SERIAL PRIMARY KEY,
   name VARCHAR(50),
   email VARCHAR(50),
   student_number VARCHAR(50),
   course VARCHAR(50),
   start_date DATE,
   end_date DATE,
   grade VARCHAR(50)
)

berdasarkan table definition, bina sql query untuk filter pelajar yang grade C keatas
""".strip()

### Llama2 7B

In [43]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': s}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

SELECT name FROM pelajar WHERE grade > 'C' </s>


### Mistral 7B

In [44]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': s}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SELECT * FROM pelajar WHERE grade >= "C"</s>


# Mathematics

From https://huggingface.co/datasets/camel-ai/math, translate using google translate.

1. Cari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|.
2. Dalam kumpulan 6 orang, setiap orang berkawan dengan sekurang-kurangnya 3 orang lain. Berapakah bilangan minimum kumpulan 3 orang yang perlu dibentuk untuk memastikan sekurang-kurangnya terdapat satu kumpulan yang kesemua ahli berkawan antara satu sama lain?
3. Cari persamaan satah yang mengandungi persilangan permukaan:\n  $x^2 + y^2 + z^2 = 25$ dan $x - y + z = 0$.

### Answer 1

Untuk mencari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|, kita perlu mencari koordinat-x bagi puncak tersebut. Puncak berlaku pada titik di mana ungkapan di dalam nilai mutlak adalah sama dengan sifar.\n\n2x - 5 = 0\n\nSekarang, selesaikan untuk x:\n\n2x = 5\nx = 5/2\n\nKoordinat-x bagi bucu ialah 5/2. Sekarang, kita perlu mencari koordinat-y yang sepadan dengan memasukkan semula koordinat-x ke dalam fungsi:\n\nf(5/2) = |2(5/2) - 5|\n\nf(5/2) = |5 - 5|\n\nf(5/2) = 0\n\nJadi, koordinat y bagi bucu ialah 0. Oleh itu, bucu bagi fungsi nilai mutlak f(x) = |2x - 5| ialah (5/2, 0).

## Answer 2

Mari kita nyatakan orang sebagai A, B, C, D, E, dan F. Kita boleh mewakili persahabatan antara mereka menggunakan graf, di mana setiap orang adalah bucu dan tepi antara dua bucu mewakili persahabatan.\n\nMemandangkan setiap orang berkawan dengan sekurang-kurangnya 3 orang lain, kita boleh mulakan dengan menyambungkan A ke B, C dan D. Sekarang, kita perlu memastikan B, C dan D mempunyai sekurang-kurangnya 3 rakan juga. Jika B berkawan dengan kedua-dua C dan D, maka kami sudah mempunyai kumpulan 3 orang yang semuanya berkawan antara satu sama lain (A, B, dan C atau A, B, dan D). Jadi, untuk meminimumkan bilangan kumpulan sedemikian, kita boleh menganggap bahawa B tidak berkawan dengan C atau D. Sebaliknya, B boleh berkawan dengan E dan F.\n\nKini, kami mempunyai persahabatan berikut: A-B, A-C, A-D, B-E dan B-F. Kita masih perlu memastikan C, D, E dan F mempunyai sekurang-kurangnya 3 rakan setiap satu. Jika C berkawan dengan D, maka kami mempunyai sekumpulan 3 orang yang semuanya berkawan antara satu sama lain (A,C, dan D). Jadi, untuk meminimumkan bilangan kumpulan sedemikian, kita boleh menganggap bahawa C tidak berkawan dengan D. Sebaliknya, C boleh berkawan dengan E dan F.\n\nKini, kami mempunyai persahabatan berikut: A-B, A-C, A-D, B-E, B-F, C-E dan C-F. Kita masih perlu memastikan D, E, dan F mempunyai sekurang-kurangnya 3 rakan setiap satu. Jika D berkawan dengan E atau F, maka kami mempunyai sekumpulan 3 orang yang semuanya berkawan antara satu sama lain (A, D, dan E atau A, D, dan F). Jadi, untuk meminimumkan bilangan kumpulan sedemikian, kita boleh menganggap bahawa D tidak berkawan dengan E atau F. Sebaliknya, D boleh berkawan dengan G.\n\nWalau bagaimanapun, kami hanya mempunyai 6 orang dalam kumpulan, jadi kami tidak boleh menambah orang baru G. Oleh itu, D mesti berkawan dengan sama ada E atau F. Tanpa kehilangan sifat umum, mari kita anggap D berkawan dengan E. Sekarang, kita mempunyai kumpulan 3 orang yang semuanya berkawan antara satu sama lain (A, D, dan E).\n\nJusteru, bilangan minimum kumpulan 3 orang yang perlu dibentuk bagi memastikan sekurang-kurangnya terdapat satu kumpulan yang kesemua ahli berkawan antara satu sama lain ialah 1.

### Llama2 7B

In [45]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Cari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|.'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Nilai bagi fungsi nilai mutlak f(x) = |2x - 5| ialah sama dengan |2x - 5| iaitu x = 3. </s>


In [46]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Dalam kumpulan 6 orang, setiap orang berkawan dengan sekurang-kurangnya 3 orang lain. Berapakah bilangan minimum kumpulan 3 orang yang perlu dibentuk untuk memastikan sekurang-kurangnya terdapat satu kumpulan yang kesemua ahli berkawan antara satu sama lain?'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

10 kumpulan </s>


In [47]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Cari persamaan satah yang mengandungi persilangan permukaan:\n  $x^2 + y^2 + z^2 = 25$ dan $x - y + z = 0$.'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Persamaan satah yang mengandungi persilangan permukaan:

$x^2 + y^2 + z^2 = 25$ dan $x - y + z = 0$ ialah

```
x1 + y1 + z1 = 25
x2 + y2 + z2 = 25
x1 + x2 + y1 + y2 + z1 + z2 = 25
```

Dalam bentuk matriks, persamaan satah ialah

```
A =
  [1 0 0 -x1 - y1 - z1]
  [1 1 -x1 - y1]
  [1 -x2 - y2]
  [1 1 -x1 - y1 - z2]
  [1 -x2 - y2]
```

Dalam bentuk matriks, persilangan permukaan ialah

```
B =
  [1]
  [0 -x1 - y1]
  [-1, -x2 - y2]
  [1]
  [-1, -x2 - y2]
  [1 -x1 - y1 - z2]
  [1 -x1 - y1 - z2]
```

Dalam bentuk matriks, persilangan permukaan ialah

```
AxB =
  [1, 0, -x1, 1, x2, -x2, y1]
  [0, -x1, 1, -x2, -x2, -y2, -x1]
  [1, -x1, -x2, -x2, -y2, -y2, -x1]
  [1, -x1, -x2, -x2, -y2, -y2, -x1]
```

Nota: Matriks A ialah matriks yang mengandungi persamaan satah dan persilangan permukaan ialah matriks yang diperoleh daripada mengendalikan A terhadap faktor penentu -1. </s>


### Mistral 7B

In [48]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Cari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|.'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Fungsi nilai mutlak mempunyai domain semua nombor sebenar dan hanya ditakrifkan untuk x sebenar.
Untuk mencari puncak, kita perlu mencari nilai maksimum atau minimum fungsi nilai mutlak.
Mengambil derivatif berkenaan dengan x, kita mempunyai:
f'(x) = 2 - 2x + 5 - 2x + 5x + 25
= 2x - 5

Menetapkan f'(x) sama dengan 0 dan menyelesaikan x, kita dapat:
2x - 5 = 0
x = 5/2

Oleh kerana terdapat lebih banyak nilai dalam domain berbanding sempadan, puncak adalah lebih rendah daripada sempadan.
Oleh itu, puncak fungsi nilai mutlak ialah $\boxed{(-\frac{5}{2}, 5/2)}$.
Jawapannya ialah: (-\frac{5}{2},5/2)</s>


In [49]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Dalam kumpulan 6 orang, setiap orang berkawan dengan sekurang-kurangnya 3 orang lain. Berapakah bilangan minimum kumpulan 3 orang yang perlu dibentuk untuk memastikan sekurang-kurangnya terdapat satu kumpulan yang kesemua ahli berkawan antara satu sama lain?'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Untuk mencari bilangan minimum 3 orang yang diperlukan untuk membentuk kumpulan, kita boleh menggunakan konsep prinsip mengira.
Jika terdapat 6 orang dalam kumpulan, maka terdapat 6C3=180 cara untuk memilih 3 orang daripada kumpulan.
Jika setiap orang berkawan dengan sekurang-kurangnya 3 orang lain, maka bilangan kumpulan 3 orang mestilah sekurang-kurangnya 180, kerana terdapat 6 orang.
Untuk memastikan bahawa tiada dua kumpulan berkongsi semua ahli, kita perlu mempunyai sekurang-kurangnya 180-1=179 ahli dalam kumpulan.
Oleh kerana setiap ahli mesti berada dalam satu atau lebih kumpulan, kita mesti mempunyai sekurang-kurangnya 179 ahli dalam kumpulan.
Oleh itu, bilangan minimum kumpulan 3 orang yang perlu dibentuk untuk memastikan sekurang-kurangnya terdapat satu kumpulan yang kesemua ahlinya berkawan antara satu sama lain ialah $\boxed{180}$.
Jawapannya ialah: 180</s>


In [50]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'Cari persamaan satah yang mengandungi persilangan permukaan:\n  $x^2 + y^2 + z^2 = 25$ dan $x - y + z = 0$.'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Kita boleh menulis semula persamaan satah dalam sebutan koordinat sebagai $(x + y)^2 + (x + z)^2 = 25$.
Ini ialah persamaan elips, dan bentuknya ialah $(x + y)^2 + (x + z)^2 = \frac{25}{4}$.
Membandingkan pekali, kita mempunyai $c = \frac{25}{4}$, iaitu jarak dari asal ke satah dalam satah Cartesan.
Persamaan satah ini diberikan oleh $x^2 + y^2 + z^2 = \frac{25}{4} + \frac{25}{4}$.
Untuk mencari persamaan satah, kita boleh menulis semula persamaan ini sebagai $(x + y)^2 + (x + z)^2 = \frac{50}{4}$, yang boleh ditulis sebagai $(x + y)^2 + (x + z)^2 = \frac{10}{2}$.
Ini ialah persamaan hiperbola, dan bentuknya ialah $(x + y)^2 + (x + z)^2 = \frac{10}{2}$.
Persamaan satah yang mengandungi titik ini ialah $\frac{10}{2} = \frac{x^2 + y^2 + z^2}{2} = \frac{10}{2}$.
Mendarab kedua-dua belah dengan 2 memberikan $10 = x^2 + y^2 + z^2$.
Jadi persamaan satah ialah $x^2 + y^2 + z^2 = \boxed{10}$.
Jawapannya ialah: 10</s>


# Translation

1. SHAH ALAM - Pertubuhan Kebajikan Anak Bersatu Selangor bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja untuk menyiasat isu kartel daging. to English
2. Smh the dude literally has a rubbish bin infront of his house. to standard Malay
3. Syed Saddiq berkata, mereka seharusnya mengingati bahawa semasa menjadi Perdana Menteri Pakatan Harapan. to English
4. Kalau yang rasa overwhelming pasal belajar CSS tu, ini 7 fasa untuk anda belajar step-by-step. 5 fasa pertama tu wajib. Cuma tak perlu mahir semua dalam setiap fasa. Dengan sedikit pemahaman dah boleh mula ke fasa seterusnya, mengikut keperluan masing2, to standard Malay

### Llama2 7B

In [52]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to English `SHAH ALAM - Pertubuhan Kebajikan Anak Bersatu Selangor bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja untuk menyiasat isu kartel daging.`'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

SHAH ALAM - The Welfare Association of United Children of Selangor agreed that the government should establish a Royal Commission of Inquiry to investigate the issue of meat cartels. </s>


In [53]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to standard Malay `Smh the dude literally has a rubbish bin infront of his house`'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Saya sedih kerana dia sebenarnya mempunyai tong sampah di hadapan rumahnya. </s>


In [54]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to English `Syed Saddiq berkata, mereka seharusnya mengingati bahawa semasa menjadi Perdana Menteri Pakatan Harapan`'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Syed Saddiq said that they should remember that when Pakatan Harapan was in power, he was the Prime Minister. </s>


In [55]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to standard Malay `Kalau yang rasa overwhelming pasal belajar CSS tu, ini 7 fasa untuk anda belajar step-by-step. 5 fasa pertama tu wajib. Cuma tak perlu mahir semua dalam setiap fasa. Dengan sedikit pemahaman dah boleh mula ke fasa seterusnya, mengikut keperluan masing2`'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Jika anda merasa bingung tentang mempelajari CSS, ini adalah 7 tahap untuk mempelajari langkah demi langkah. Hanya 5 tahap pertama diperlukan. Walaupun tidak perlu mahir dalam setiap fasa, dengan sedikit pemahaman, anda boleh mulai ke fasa berikutnya, bergantung pada keperluan anda. </s>


### Mistral 7B

In [56]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to English `SHAH ALAM - Pertubuhan Kebajikan Anak Bersatu Selangor bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja untuk menyiasat isu kartel daging.`'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SHAH ALAM - The People's Welfare Organization of Selangor agreed to the government creating a Royal Commission of Inquiry to investigate the meat cartel issue.</s>


In [58]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to standard Malay `Smh the dude literally has a rubbish bin infront of his house`'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Menggelengkan kepala, orang itu benar-benar mempunyai tong sampah yang sangat buruk di depannya.</s>


In [59]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to English `Syed Saddiq berkata, mereka seharusnya mengingati bahawa semasa menjadi Perdana Menteri Pakatan Harapan`'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Syed Saddiq stated that they should remember that when they were Prime Minister of Pakatan Harapan</s>


In [61]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'translate to standard Malay `Kalau yang rasa overwhelming pasal belajar CSS tu, ini 7 fasa untuk anda belajar step-by-step. 5 fasa pertama tu wajib. Cuma tak perlu mahir semua dalam setiap fasa. Dengan sedikit pemahaman dah boleh mula ke fasa seterusnya, mengikut keperluan masing2`'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Jika anda berasa terlalu terharu semasa belajar CSS, berikut adalah 7 fasa yang perlu anda pelajari langkah demi langkah. 5 fasa pertama adalah wajib. Tidak perlu mahir dalam setiap fasa, tetapi cukup untuk memahami asasnya sebelum meneruskan ke fasa seterusnya. Sesuaikan pembelajaran anda dengan keperluan anda.</s>


# RAG

```
KUALA LUMPUR: Perdana Menteri menegaskan terdapat had tertentu yang perlu dipatuhi oleh kerajaan dalam isu kewarganegaraan kerana ia melibatkan Majlis Raja-Raja.

Datuk Seri Anwar Ibrahim berkata, kerajaan perlu mengambil kira dan menghormati pandangan Majlis Raja-Raja berhubung isu pemberian kerakyatan tersebut.

“Kita akan maklumkan kepada mereka (Ahli Parlimen) kerana ini telahpun dibentangkan kepada Majlis Raja-Raja. Terdapat had mengenai apa yang kita boleh lakukan selepas perbincangan (dilakukan).

“Persidangan Majlis Raja-Raja sudah mengambil pendirian (mengenai isu kewarganegaraan), jadi kita perlu menghormatinya,” katanya kepada media selepas melancarkan Ekspo Digital Malaysia (MDX) 2023 di Pusat Pameran Perdagangan Antarabangsa Malaysia (MITEC).

Turut hadir Menteri Komunikasi dan Digital, Fahmi Fadzil dan Timbalannya, Teo Nie Ching.

Terdahulu media melaporkan, seorang Ahli Parlimen DAP memberitahu terdapat kemungkinan ahli-ahli Parlimen tidak sefahaman dan akan menentang arahan Ketua Whip di Dewan Rakyat, jika kerajaan tidak menangani dengan baik isu lapan cadangan pindaan Perlembagaan Persekutuan berkaitan kewarganegaraan.  
```

from https://www.utusan.com.my/nasional/2023/11/isu-kewarganegaraan-kerajaan-perlu-hormat-pandangan-majlis-raja-raja/

In [63]:
context = """
KUALA LUMPUR: Perdana Menteri menegaskan terdapat had tertentu yang perlu dipatuhi oleh kerajaan dalam isu kewarganegaraan kerana ia melibatkan Majlis Raja-Raja.

Datuk Seri Anwar Ibrahim berkata, kerajaan perlu mengambil kira dan menghormati pandangan Majlis Raja-Raja berhubung isu pemberian kerakyatan tersebut.

“Kita akan maklumkan kepada mereka (Ahli Parlimen) kerana ini telahpun dibentangkan kepada Majlis Raja-Raja. Terdapat had mengenai apa yang kita boleh lakukan selepas perbincangan (dilakukan).

“Persidangan Majlis Raja-Raja sudah mengambil pendirian (mengenai isu kewarganegaraan), jadi kita perlu menghormatinya,” katanya kepada media selepas melancarkan Ekspo Digital Malaysia (MDX) 2023 di Pusat Pameran Perdagangan Antarabangsa Malaysia (MITEC).

Turut hadir Menteri Komunikasi dan Digital, Fahmi Fadzil dan Timbalannya, Teo Nie Ching.

Terdahulu media melaporkan, seorang Ahli Parlimen DAP memberitahu terdapat kemungkinan ahli-ahli Parlimen tidak sefahaman dan akan menentang arahan Ketua Whip di Dewan Rakyat, jika kerajaan tidak menangani dengan baik isu lapan cadangan pindaan Perlembagaan Persekutuan berkaitan kewarganegaraan.  
"""

### Llama2 7B

In [64]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, isu apa ni'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Perdana Menteri menegaskan terdapat had tertentu yang perlu dipatuhi oleh kerajaan dalam isu kewarganegaraan kerana ia melibatkan Majlis Raja-Raja. </s>


In [65]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, apa yang dilancarkan'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Ekspo Digital Malaysia (MDX) 2023 di Pusat Pameran Perdagangan Antarabangsa Malaysia (MITEC) </s>


In [71]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, sapa yang mai dalam pelancaran tu'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer_llama([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = llama.generate(**generate_kwargs)
print(tokenizer_llama.decode(r[0]).split('[/INST]')[1].strip())

Perdana Menteri menegaskan bahawa kerajaan perlu mengambil kira dan menghormati pandangan Majlis Raja-Raja berhubung isu pemberian kerakyatan. </s>


### Mistral 7B

In [69]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, isu apa ni'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Isu yang dibincangkan adalah had tertentu yang perlu dipatuhi oleh kerajaan dalam isu kewarganegaraan kerana ia melibatkan Majlis Raja-Raja.</s>


In [70]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, apa yang dilancarkan'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Datuk Seri Anwar Ibrahim telah melancarkan Ekspo Digital Malaysia (MDX) 2023 di Pusat Pameran Perdagangan Antarabangsa Malaysia (MITEC).</s>


In [72]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': f'paragraph `{context}`, sapa yang mai dalam pelancaran tu'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer_mistral([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = mistral.generate(**generate_kwargs)
print(tokenizer_mistral.decode(r[0]).split('[/INST]')[1].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Menteri Komunikasi dan Digital, Fahmi Fadzil dan Timbalannya, Teo Nie Ching.</s>
